1. IMPORT LIBRARY YANG DIPERLUKAN

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

2. EKSTRAKSI FITUR DATA GSR

In [ ]:
# Fungsi untuk memproses data dan mengekstrak fitur
def process_gsr_file(input_file_path, output_file_path):
    data = pd.read_csv(input_file_path)

    # Pastikan 'Timestamp' diubah ke datetime
    try:
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    except Exception as e:
        print(f"Timestamp tidak dalam format datetime untuk file {input_file_path}. Melanjutkan dengan format asli.")

    # Ekstraksi fitur untuk setiap baris data
    results = []
    for i in range(len(data)):
        subset = data.iloc[:i + 1]

        mean_scl = np.mean(subset['Filtered_Conductance'])
        max_scl = np.max(subset['Filtered_Conductance'])

        # Deteksi puncak SCR
        peaks, _ = find_peaks(subset['Filtered_Conductance'], height=0.05)
        max_scr_peak = subset['Filtered_Conductance'].iloc[peaks].max() if len(peaks) > 0 else 0

        rise_time_intervals = np.diff(peaks)
        std_dev_rise_time = np.std(rise_time_intervals) if len(rise_time_intervals) > 1 else 0

        num_peaks = len(peaks)

        # Simpan hasil ekstraksi untuk baris terakhir subset
        results.append({
            'Timestamp': subset['Timestamp'].iloc[-1],
            #'Label': subset['Label'].iloc[-1],  # Menyertakan label dari data gabungan
            'Mean SCL': mean_scl,
            'Max SCL': max_scl,
            'Max SCR Peak': max_scr_peak,
            'Std Dev SCR Rise Time': std_dev_rise_time,
            'Number of SCR Peaks': num_peaks
        })
        
        # Menyimpan hasil ekstraksi ke dalam DataFrame baru dan file CSV
    results_df = pd.DataFrame(results)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)  # Membuat folder jika belum ada
    results_df.to_csv(output_file_path, index=False)
    print(f"Data ekstraksi untuk {input_file_path} telah disimpan ke {output_file_path}.")

3. MEMUAT FILE DATA YANG AKAN DIEKSTRAKSI FITUR SESUAI ISI FOLDER DAN MENYIMPAN CSV BARU

In [ ]:
# Path folder input dan output
input_base_folder = 'E:/SKRIPSI/Dataset/data uji/'  # Folder utama input
output_base_folder = 'E:/SKRIPSI/Dataset/Data Uji Fitur GSR/'  # Folder utama output

# Daftar sub-folder sesi
sessions = ['sesi 1', 'sesi 2', 'sesi 3', 'sesi 4']

# Iterasi untuk setiap sesi
for session in sessions:
    input_file_path = os.path.join(input_base_folder, 'Reza', session, 'gsr_data_processed.csv')  # Lokasi file input
    output_file_path = os.path.join(output_base_folder, session.capitalize(), 'Reza_features.csv')  # Lokasi file output

    # Periksa apakah file input ada
    if os.path.exists(input_file_path):
        print(f"Memproses file: {input_file_path}")
        process_gsr_file(input_file_path, output_file_path)
    else:
        print(f"File {input_file_path} tidak ditemukan. Lewati sesi ini.")